<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/LogisticRegression/LR_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classification by Logistic Regression and Count Vectorizer**

----------

 import library

  إستيراد المكتبات

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk
nltk.download('stopwords')
from nltk.stem.isri import ISRIStemmer
import re
import string
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--------

**1- Import Data set**

In [2]:
# df = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/3.xlsx")
df = pd.read_excel("drive/MyDrive/mining/4.xlsx")
df.head()

,text,targe
0,أكد وزير الاتصال الناطق الرسمي باسم الحكومة مص...,0
1,عبدالاله بوسحابة أخبارنا المغربية أثار فيلم أل...,0
2,عبدالاله بوسحابة اخبارنا المغربية أبدى القائمو...,0
3,أخبارنا المغربية اختار الفنان المغربي البشير ع...,0
4,المادة الأولى على ملك غرناطة القادة الفقهاء ال...,0


------
shuffle  
بعثرة البيانات

In [3]:
df=shuffle(df)
df

,text,targe
4018,توفي صباح اليوم الأحد شاب ينحدر من دوار ابراحن...,1
57092,تعرض للضرب لمنعه من مستحقاته المالية ومحكمة ال...,1
47578,الناشط الأمازيغي قال إن فتوى حد الردة لا تليق ...,3
54010,أعلنت لجنة دعم الإنتاج السينمائي التي عقدت دور...,0
24475,قال نزار بركة أن الحوار الاجتماعي كلف مليار در...,2
...,...,...
58820,جرأة في تنزيل القرارات وتكريس منطق الاستمرارية...,2
56119,هاجم مختل عقليا صباح الأربعاء الماضي فاعلة جمع...,1
31388,حملت مباراة فريق اتحاد طنجة ضد نظيره الدفاع ال...,4
46413,سلط رئيس المجلس الاقتصادي والاجتماعي والبيئي ن...,3


------
**2- Clean Data**

تنظيف البيانات

In [4]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)


378
0
text     True
targe    True
dtype: bool
68690
                                                    text  targe
4018   توفي صباح اليوم الاحد شاب ينحدر من دوار ابراحن...      1
57092  تعرض لضرب لمنعه من مستحقاته الماليه ومحكمه الا...      1
47578  ناشط الامازيغي قال ان فتوي حد الرده لا تليق بع...      3
54010  اعلنت لجنه دعم الانتاج السينمائي التي عقدت دور...      0
24475  قال نزار بركه ان الحوار الاجتماعي كلف مليار در...      2
...                                                  ...    ...
58820  جراه في تنزيل القرارات وتكريس منطق الاستمراريه...      2
56119  هاجم مختل عقليا صباح الاربعاء الماضي فاعله جمع...      1
31388  حملت مباراه فريق اتحاد طنجه ضد نظيره الدفاع ال...      4
46413  سلط رئيس المجلس الاقتصادي والاجتماعي والبيئي ن...      3
55315  ادانه شابين وزوجتين بشيشاوهادانت الهياه القضائ...      1

[68312 rows x 2 columns]


---------
accuracy calculation function

عمل تابع لحساب الدقة

In [5]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

----------------------
**3- Feature Extraction:** Count Vectorizer

استخراج الميزات(تحويل النص إلى أرقام)


**4- classification:** Logistic Regression

تصنيف البيانات

with Cross validation

لتوزيع عينات الاختبار والتأكد من عدم وجود

Overfitting

In [6]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold


x =clean_df['text'].values
y = clean_df['targe'].values

strtfdKFold = StratifiedKFold(n_splits=4)
kfold = strtfdKFold.split(x, y)

classifier = LogisticRegression()
vectorizer = CountVectorizer()
result_LogisticRegressionClassifier=[]

for k, (train_index, test_index) in enumerate(kfold):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    vectorizer.fit(x_train)

    X_train = vectorizer.transform(x_train)
    X_test  = vectorizer.transform(x_test)
    X_train
    model=classifier.fit(X_train, y_train)
    y_pred=(model.predict(X_test))
    result_LogisticRegressionClassifier.append(calculate_results(y_test, y_pred))

result_LogisticRegressionClassifier

CPU times: user 7min 36s, sys: 2min 22s, total: 9min 59s
Wall time: 7min 15s


حساب متوسط النتائج للـ5 مراحل من

 cross validation 

In [7]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_LogisticRegressionClassifier:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_LogisticRegressionClassifier)
f1=f1/len(result_LogisticRegressionClassifier)
precision=precision/len(result_LogisticRegressionClassifier)
recall=recall/len(result_LogisticRegressionClassifier)
# (result_DecisionTreeClassifier)
results_LogisticRegressionClassifier=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_LogisticRegressionClassifier

[{'accuracy': 94.39629933247453,
  'f1': 0.9439955048810575,
  'precision': 0.9440560220807676,
  'recall': 0.9439629933247453}]

--------
-----
---------
**5- Show Results**

النتائج

نتائج التدريب لكل قاعدة بيانات

In [ ]:
print('LogisticRegressionClassifier results in dataset 1:')
results_LogisticRegressionClassifier

LogisticRegressionClassifier results in dataset 1:


[{'accuracy': 96.35964339589191,
  'f1': 0.9636438504186007,
  'precision': 0.963822932097349,
  'recall': 0.9635964339589191}]

In [ ]:
print('LogisticRegressionClassifier results in dataset 2:')
results_LogisticRegressionClassifier

LogisticRegressionClassifier results in dataset 2:


[{'accuracy': 96.30527652695993,
  'f1': 0.9630648988998682,
  'precision': 0.9631047401343221,
  'recall': 0.9630527652695993}]

In [ ]:
print('LogisticRegressionClassifier results in dataset 3:')
results_LogisticRegressionClassifier

LogisticRegressionClassifier results in dataset 3:


[{'accuracy': 94.83628470244224,
  'f1': 0.948372544393749,
  'precision': 0.9484109573769226,
  'recall': 0.9483628470244223}]

In [ ]:
print('LogisticRegressionClassifier results in dataset 4:')
results_LogisticRegressionClassifier

LogisticRegressionClassifier results in dataset 4:


[{'accuracy': 94.35823866963344,
  'f1': 0.9436339010932193,
  'precision': 0.9437197486968766,
  'recall': 0.9435823866963345}]

In [8]:
Final_Result=[
              {'accuracy': 96.35964339589191,
  'f1': 0.9636438504186007,
  'precision': 0.963822932097349,
  'recall': 0.9635964339589191}
  ,
  {'accuracy': 96.30527652695993,
  'f1': 0.9630648988998682,
  'precision': 0.9631047401343221,
  'recall': 0.9630527652695993}
  ,  
  {'accuracy': 94.83628470244224,
  'f1': 0.948372544393749,
  'precision': 0.9484109573769226,
  'recall': 0.9483628470244223}
  ,
  {'accuracy': 94.35823866963344,
  'f1': 0.9436339010932193,
  'precision': 0.9437197486968766,
  'recall': 0.9435823866963345}
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
print("RAM: 0.79 ,CPU times: user 7min 36s  ")
print('Final_Result:')
(Final_Result)

RAM: 0.79 ,CPU times: user 7min 36s  
Final_Result:


[{'accuracy': 95.46486082373187,
  'f1': 0.9546787987013593,
  'precision': 0.9547645945763676,
  'recall': 0.9546486082373189}]